In [208]:
# -*- encoding: utf-8 -*-
import sys
import codecs
import csv
import MeCab
import os

In [209]:
reload(sys)
sys.setdefaultencoding('utf-8')

In [210]:
import re, pprint
def pp(obj):
    pp = pprint.PrettyPrinter(indent=4, width=160)
    str = pp.pformat(obj)
    return re.sub(r"\\u([0-9a-f]{4})", lambda x: unichr(int("0x"+x.group(1),16)), str)

In [211]:
# 读取停用词一览
def readStopWord(ssfile):
    ss = []
    fr = open(ssfile,"r")
    for line in fr.readlines():
        line = line.strip()
        if line != '':
            ss.append(line)
    fr.close()
    return ss

In [212]:
# 读取CSV文件
def readDataFile(fileName):
    with open(fileName,"r+") as csvfile:
        reader = csv.reader(csvfile)
        # skip header
#         header = next(reader)   
#         print "Read article:",fileName
        # 读取内容
        for line in reader:
#             print pp(line)
            return line

In [213]:
# 获取词汇本体
def get_surfaces(node):
    words = []
    while node:
#         word  = node.surface.decode('utf-8')
        word  = node.surface
        words.append(word)
        node = node.next
    return words

In [214]:
# 获取词性
def get_nouns(node):
    nouns = []
    while node:
#         noun = node.feature.split(",")[0].decode('utf-8')
        noun = node.feature.split(",")[0]
        nouns.append(noun)
        node = node.next
    return nouns

In [215]:
# 利用词性筛选词汇
def select_feature(data,attr):
    result = []
    pattern1 = re.compile(r'[^0-9]+')
    pattern2 = re.compile(ur'[^０１２３４５６７８９]+')
    pattern3 = re.compile(ur'[^\[\]!$%&\'\"\(\):\-\.,/;=]+')
    pattern4 = re.compile(ur'[^！　％、。）※「」（＞～』＜？－．♪【⇒∞★〇・]+')
    for (w,a) in data:
        if a in attr:
#             print w,a
            # 过滤所有数字
            tmp = w.decode('utf-8')
            matcher1 = re.match(pattern1,w)
            matcher2 = re.match(pattern2,tmp)
            matcher3 = re.match(pattern3,tmp)
            matcher4 = re.match(pattern4,tmp)
            if (matcher1 is not None) and (matcher2 is not None) and (matcher3 is not None) and (matcher4 is not None):
                result.append((w,a))
    return result

In [216]:
# 把筛选以后的结果写入CSV文件
def writeOutput(fileName,data,mode):
    with open(fileName,mode) as csvfile: 
        writer = csv.writer(csvfile)
        
#         line = ""
#         for w in data:
#             line = line + w + "\t"
#         writer.writerows( line  + "\n")
        
        writer.writerows(data)

In [217]:
# 输出结果有两个文件
# 全量分词结果文件：ALLWORD_filename
# 词性筛选结果文件：mode_file (默认筛选名词 mode = "N")
# 筛选 形容詞 ：mode 指定 "A"
# 筛选 动词 ：mode 指定 "V"
# 例如：筛选 名词，形容词的场合 :mode = "NA"
def processFile(inPath,outPath,stopwords,mode="N"):
    
    # 获取文件列表
    fileList = os.listdir(inPath)
    
    for f in fileList:
        # 编辑文件路径
        fileName = inPath + f
        # 获取文件内容
        content = readDataFile(fileName)
        
    
        # 定义分词器
        mt = MeCab.Tagger('mecabrc')
#         node = mt.parseToNode(content[1].encode('utf-8'))
        node = mt.parseToNode(content[0])
        words, nouns = get_surfaces(node), get_nouns(node)
        wordList = zip(words,nouns)
        
#         # 全词分词结果写入CSV
#         outFile_allword = outPath + "ALLWORD_" + f
#         writeOutput(outFile_allword,wordList,"wb")
  
        # 编辑筛选词性的分词输出文件名
        outFile_noun = outPath + mode + "_" + f
        
        attr = [u'名詞']
        
        if "A" in mode:
            # 筛选形容詞结果写入CSV
            attr.append(u'形容詞')
  
        if "V" in mode:
            # 筛选動詞结果写入CSV
            attr.append(u'動詞')
        
        # 筛选名词结果写入CSV
        selectList = select_feature(wordList,attr)
        outWord = []
        for (w,a) in selectList: 
            if w not in stopwords:
                outWord.append((w,a))
#                 outWord.append(w)
#                 print w
#         writeOutput(outFile_noun,outWord,"wb")
        writeOutput(outPath,outWord,"a")

In [218]:
def main():
    # 修改抓取的文本文件路径
    input_path = "/home/hadoop/DataSencise/text_data/comment/"
    # 修改分词结果文件路径
    output_path = "/home/hadoop/DataSencise/text_data/test/test.txt"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    ssfile = "./conf/Japanese_stopwords.txt"
    ssList = readStopWord(ssfile)
    # 筛选名词    
#     processFile(input_path,output_path,"N")
    # 筛选名词和形容次
#     processFile(input_path,output_path,"NA")
    # 筛选名词和形容次，动词
    processFile(input_path,output_path,ssList,"NA")

In [219]:
if __name__ == '__main__':
    main()